# Russian entity-based sentiment analysis using BERT

Попробуем классифицировать по CLS токену, а на само слово навешивать маску.


In [1]:
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt

--2021-12-26 18:53:05--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228391 (223K) [text/plain]
Saving to: ‘train_aspects.txt’

train_aspects.txt   100%[===================>] 223.04K  --.-KB/s    in 0.02s   

2021-12-26 18:53:05 (8.97 MB/s) - ‘train_aspects.txt’ saved [228391/228391]

--2021-12-26 18:53:06--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting respon

In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 31.5 MB/s 
     |████████████████████████████████| 596 kB 34.1 MB/s 
     |████████████████████████████████| 3.3 MB 41.7 MB/s 
     |████████████████████████████████| 61 kB 135 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bs4
import os
from tqdm.notebook import tqdm
sns.set(font_scale=1.4)

import torch
from torch import nn
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from transformers import AutoModel, AutoTokenizer, AdamW,\
                        get_linear_schedule_with_warmup

In [4]:
aspects = pd.read_csv('train_aspects.txt', sep='\t', header=None)
aspects.columns = ['sentence_id', 'tag', 'term', 'from', 'to', 'sentiment']
reviews = pd.read_csv('train_reviews.txt',sep='\t', header=None)
reviews.columns = ['sentence_id', 'text']

In [5]:
data_train = aspects.merge(reviews, on='sentence_id')

Загрузим данные:

In [6]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [7]:
data_train.head()

,sentence_id,tag,term,from,to,sentiment,text
0,3976,Whole,ресторане,71,80,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
1,3976,Whole,ресторанах,198,208,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
2,3976,Whole,ресторане,256,265,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
3,3976,Service,Столик бронировали,267,285,neutral,"День 8-го марта прошёл, можно и итоги подвести..."
4,3976,Service,администратор,322,335,positive,"День 8-го марта прошёл, можно и итоги подвести..."


In [8]:
data_train['sentiment'].unique()

array(['neutral', 'positive', 'both', 'negative'], dtype=object)

Создадим датасет:

In [9]:
sentiment_vocab = {'negative': 0, 'positive': 1, 'neutral': 2, 'both': 3}

class Dataset():

    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        text = self.data['text'].values[index]
        sentiment = self.data['sentiment'].values[index]
        masked_text = (
            text[:self.data['from'].values[index]] + 
            '[MASK]' + 
            text[self.data['to'].values[index]:]
        )
        return masked_text, sentiment_vocab[sentiment]

    def __len__(self):
        return len(self.data)

In [10]:
train_data, val_data = train_test_split(data_train, test_size=0.25)

train_dataset = Dataset(train_data)
val_dataset = Dataset(val_data)
# test_dataset = Dataset(data_test)

Загрузим BERT:

In [11]:
# Store the model we want to use
MODEL_NAME = "bert-base-multilingual-cased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(
    MODEL_NAME
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, 
    do_lower_case=True
)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Зададим классификатор:

In [12]:
class Classifier(nn.Module):

    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=0.3)
        #self.linear_1 = nn.Linear(312, 4)
        self.linear_1 = nn.Linear(768, 4)
        self.softmax = nn.Softmax()

    def forward(self, x, attention_mask=None):
        output  = self.bert(
            x, attention_mask=attention_mask
        )
        x = output['last_hidden_state'][:, 0]
        x = self.dropout(x)
        x = self.linear_1(x)
        x = self.softmax(x)
        return x

Зададим условия для тренировки:

In [13]:
model_classifier = Classifier(model).cuda()

n_epoch = 10
batch_size = 8

train_loss = []
val_loss = []

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = AdamW(
    model.parameters(),
    weight_decay =  1e-4, 
    lr = 2e-5, 
    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
)

train_scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps = 0, # Default value in run_glue.py
    num_training_steps = len(train_dataloader) * n_epoch
)

Начнем тренировку модели:

In [15]:
for epoch in tqdm(range(n_epoch)):
    model_classifier.train()
    current_train_loss = 0
    for text, y in tqdm(train_dataloader, leave=False):
        tokens = tokenizer.batch_encode_plus(
            list(text),
            pad_to_max_length=True,
            return_tensors='pt'
        )

        logits = model_classifier(
            tokens['input_ids'].cuda(),
            attention_mask=tokens['attention_mask'].cuda()            
        )

        loss = criterion(logits, y.cuda())
        current_train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        # избавимся от взрывающихся градиентов
        nn.utils.clip_grad_norm_(
            model_classifier.parameters(), 
            1.0
        )

        optimizer.step()
        train_scheduler.step()
        

    train_loss.append(current_train_loss / len(train_dataloader))

    model_classifier.eval()
    current_val_loss = 0
    predictions = []
    actuals = []
    for text, y in val_dataloader:

        tokens = tokenizer.batch_encode_plus(
            list(text),
            pad_to_max_length=True,
            return_tensors='pt'
        )

        logits = model_classifier(
            tokens['input_ids'].cuda(),
            attention_mask=tokens['attention_mask'].cuda()    
        )
        loss = criterion(logits, y.cuda())
        current_val_loss += loss.item()

        predictions += list(logits.argmax(axis=1).data.cpu().numpy())
        actuals += list(y.data.cpu().numpy())

    print('Accuracy:', accuracy_score(actuals, predictions))    
    val_loss.append(current_val_loss / len(val_dataloader))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/447 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: ignored

In [ ]:
from sklearn.metrics import accuracy_score

Посмотрим, как вел себя лосс в процессе обучения:

In [ ]:
plt.figure(figsize=(16, 8))

plt.plot(train_loss, label='Train')
plt.plot(val_loss, label='Val')
plt.title('Ошибка')
plt.xlabel('Эпоха')
plt.ylabel('Cross Entropy Loss')
plt.legend()

plt.show()

In [ ]:
torch.save(
    model_classifier.state_dict(), 
    '/content/drive/My Drive/NLP/dialog-sent/models/bert-base/entity-classifier-masked'
)

In [ ]:
model_classifier.load_state_dict(
    torch.load(
        '/content/drive/My Drive/NLP/dialog-sent/models/bert-base/entity-classifier-masked'
    )
)

Посомтрим на точность:

In [ ]:
model_classifier.cuda();

In [ ]:
test_dataloader = DataLoader(
    test_dataset, batch_size=3
)


predictions = []
actuals = []

model_classifier.eval()

for text, y in tqdm(test_dataloader):

    tokens = tokenizer.batch_encode_plus(
        text,
        pad_to_max_length=True,
        return_tensors='pt'
    )

    logits = model_classifier(
        tokens['input_ids'].cuda(),
        attention_mask=tokens['attention_mask'].cuda()   
    ).argmax(dim=1)
    
    predictions += list(logits.data.cpu().numpy())
    actuals += list(y.data.cpu().numpy())

In [ ]:
print('F1_macro:', f1_score(actuals, predictions, average='macro'))
print('Accuracy:', accuracy_score(actuals, predictions))